# Chapter 3b
Randomized SVD: 
- **fast**. 
- **Gives us truncated SVD** (traditional SVD throw away small singular values and their corresponding columns). 

Two keys are: 
- Often useful to reduce dimension of data in a way that preserves distances. Johnson-Lindenstrauss lemma is a classic result of this type. 
- No better general SVD method yet, so we use smaller matrix only. 

Below is over-simplified version of `randomized_svd` (core ideas). Main part is we multiply our original matrix by smaller random matrix (`M @ rand_matrix`) to produce `smaller_matrix`, and use same `np.linalg.svd` as before. 

In [1]:
def randomized_svd(M, k=10):
    m, n = M.shape
    transpose = False
    if m < n: 
        transpose = True
        M = M.T

    rand_matrix = np.random.normal(size=(M.shape[1], k))  # short side of k
    Q, _ = np.linalg.qr(M @ rand_matrix, mode="reduced")  # long side of k
    smaller_matrix = Q.T @ M
    U_hat, s, V = np.linalg.svd(smaller_matrix, full_matrices=False)
    U = Q @ U_hat

    if transpose: return V.T, s.T, U.T
    else: return U, s, V

**Bayes theorem** useful for false positives and false negatives. 

## Numerical Stability
We do $x_1 = f(\frac{1}{10})$, and we continue with $x_2 = f(x_1)$ and keep going for many iterations. See what happens. 

In [2]:
def f(x): 
    if x < 1/2: return 2 * x
    else: return 2 * x - 1

In [3]:
x = 1/10
for i in range(80): 
    print(x)
    x = f(x)

0.1
0.2
0.4
0.8
0.6000000000000001
0.20000000000000018
0.40000000000000036
0.8000000000000007
0.6000000000000014
0.20000000000000284
0.4000000000000057
0.8000000000000114
0.6000000000000227
0.20000000000004547
0.40000000000009095
0.8000000000001819
0.6000000000003638
0.2000000000007276
0.4000000000014552
0.8000000000029104
0.6000000000058208
0.20000000001164153
0.40000000002328306
0.8000000000465661
0.6000000000931323
0.20000000018626451
0.40000000037252903
0.8000000007450581
0.6000000014901161
0.20000000298023224
0.4000000059604645
0.800000011920929
0.6000000238418579
0.20000004768371582
0.40000009536743164
0.8000001907348633
0.6000003814697266
0.20000076293945312
0.40000152587890625
0.8000030517578125
0.600006103515625
0.20001220703125
0.4000244140625
0.800048828125
0.60009765625
0.2001953125
0.400390625
0.80078125
0.6015625
0.203125
0.40625
0.8125
0.625
0.25
0.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


Math is continuous and infinite, but computers are discrete and finite. Limitations of number representations (mantissa and exponent):  
1) They can't be arbitrarily large or small. 
2) There must be gaps between them. 

It's possible to create calculations that give very wrong answers (particularly repeating operation many times, since each operation could multiply error.)

IEEE Double precision arithmetic allows number as large as $1.79 \times 10^{308}$ and as small as $2.23 \times 10^{-308}$. Floats and doubles are not equidistant. 

### Machine Epsilon
Half the distance between 1 and next large number can vary by computer. IEEE standards for double precision specify: 
$$ \epsilon_{\text{machine}} = 2^{-53} \approx 1.11 \times 10^{-16} $$

Two important properties of FP arithmetic: 
- Difference between real number $x$ and closest FP approx $fl(x)$ is always smaller than $\epsilon_{\text{machine}}$ in relative terms. FOr some $\epsilon$, where $|\epsilon| \leq \epsilon_{\text{machine}}$: 
$$ fl(x) = x \cdot (1 + \epsilon) $$
- Where is any operation ($+, -, \times, \div$), and $\otimes$ is its *floating point analogue*: 
$$ x \otimes y = (xy)(1 + \varepsilon) $$
for some $\varepsilon$, where $|\varepsilon| \leq \varepsilon_{\text{machine}}$. Every FP arithmetic is exact up to a relative error of size $\varepsilon_{\text{machine}}$. 

## Speed of different types of memory: 
| What | Speed (ns) |
| ---- | ---------- |
| L1 cache reference | 0.5 |
| L2 cache reference | 7 |
| Main memory reference/RAM | 100 |
| Send 2K bytes over 1 Gbps network | 20 µs |
| Read 1 MB sequentially from memory | 250 µs |
| Round trip within same datacenter | 500 µs |
| Disk seek | 10 ms |
| Read 1 MB sequentially from network | 10 ms |
| Read 1 MB sequentially from disk | 30 ms |
| Send packet CA -> Netherlands -> CA | 150 ms |

Check the updated [interactive version here](https://colin-scott.github.io/personal_website/research/interactive_latency.html).

Each successive memory type is (at least) an order of magnitude worse than the one before it. Disk seeks are **very slow**. 

# Regex

In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
from pathlib import Path
import pandas as pd
import re
path = Path("input")

In [6]:
df = pd.read_csv(path/"Austin_Public_Health_Locations.csv")
df.head()

,Facility Name,Street Address,Zip Code,Hours,Website,Phone Number,Other Phone,Building ID,Ownership Status,Owner,Occupying Division,Occupancy Type,Sq. Ft.,Year Built
0,Bastrop WIC Clinic,"443 Texas Highway 71\nBastrop, Texas 78602\n(3...",78602,"Monday 7:30am to 7pm, closed 12 noon to 1pm; T...",NaN,512-972-4942,NaN,BAS,Lease,The Marketplace at Bastrop,Community Services,Clinic,1400.0,NaN
1,Blackland Neighborhood Center,"2005 Salina St\nAustin, Texas 78722\n(30.28075...",78722,Monday to Thursday 8am to 6pm; Friday 8am to 1...,NaN,512-972-5790,NaN,BNC,Own,City of Austin,Community Services,"Neighborhood Center, Offices",347.0,1984.0
2,Clarksville Community Health Center,"1000 Toyath Street\nAustin, Texas 78703\n(30.2...",78703,"Monday to Friday, 8am - 4:30pm",http://www.austintexas.gov/department/clarksvi...,NaN,NaN,CNC,Own,City of Austin,Disease Prevnetion and Health Promotion,"Clinic, Offices",3500.0,NaN
3,Del Valle WIC Clinic,"3518 FM 973\nDel Valle, Texas 78617\n(30.19879...",78617,Monday 8am to 7pm; closed 12 noon to 12:30pm; ...,http://www.austintexas.gov/department/del-vall...,512-972-4942,NaN,DEL,NaN,Travis County,Community Services,Clinic,1000.0,NaN
4,East Austin Neighborhood Center,"211 Comal St\nAustin, Texas 78702\n(30.259718,...",78702,Monday toThursday 8am to 6pm; Friday 8am to noon,http://www.austintexas.gov/department/east-aus...,512-972-6650,NaN,EAN,Own,City of Austin,Community Services,"Clinic, Neighborhood Center",4304.0,1981.0


First, read database into raw text string. 

In [7]:
with open(path/"Austin_Public_Health_Locations.csv", "r") as file:
    data = file.read().replace("\n", "")

### Extract the phone numbers
Constructing regex to match phone numbers and break them into tuples. This involves trial and error. 

In [8]:
re_extract_phone_number = re.compile(r"(\d\d\d)-(\d+)-(\d+)")

In [9]:
# from IPython import display
phone_number_list = re_extract_phone_number.findall(data)
phone_number_list

[('512', '972', '4942'),
 ('512', '972', '5790'),
 ('512', '972', '4942'),
 ('512', '972', '6650'),
 ('512', '972', '4784'),
 ('512', '972', '4942'),
 ('512', '972', '4942'),
 ('512', '972', '4942'),
 ('512', '972', '4942'),
 ('512', '972', '4942'),
 ('512', '719', '3010'),
 ('800', '514', '6667'),
 ('512', '962', '6650'),
 ('512', '972', '4942'),
 ('512', '972', '4942'),
 ('512', '972', '4942'),
 ('512', '972', '4942'),
 ('512', '972', '5400'),
 ('512', '972', '6740'),
 ('512', '972', '4942'),
 ('512', '978', '0300'),
 ('512', '972', '6840'),
 ('512', '972', '4942'),
 ('512', '972', '5139'),
 ('512', '972', '4942'),
 ('512', '972', '5010'),
 ('512', '978', '9740'),
 ('512', '972', '5000'),
 ('512', '972', '4100'),
 ('512', '972', '5000')]

In [10]:
# Join together separated by space: 
[" ".join(pn) for pn in phone_number_list]

['512 972 4942',
 '512 972 5790',
 '512 972 4942',
 '512 972 6650',
 '512 972 4784',
 '512 972 4942',
 '512 972 4942',
 '512 972 4942',
 '512 972 4942',
 '512 972 4942',
 '512 719 3010',
 '800 514 6667',
 '512 962 6650',
 '512 972 4942',
 '512 972 4942',
 '512 972 4942',
 '512 972 4942',
 '512 972 5400',
 '512 972 6740',
 '512 972 4942',
 '512 978 0300',
 '512 972 6840',
 '512 972 4942',
 '512 972 5139',
 '512 972 4942',
 '512 972 5010',
 '512 978 9740',
 '512 972 5000',
 '512 972 4100',
 '512 972 5000']

# Revisiting Naïve Bayes in Excel Spreadsheet. 

In [63]:
from fastai.text.all import *
from utils import *

In [64]:
path = untar_data(URLs.IMDB_SAMPLE)
df = pd.read_csv(path/"texts.csv")

imdb_clas = DataBlock(
    blocks=(TextBlock.from_df("text", seq_len=72), CategoryBlock),
    get_x=ColReader("text"), get_y=ColReader("label"), splitter=ColSplitter()
)

dls = imdb_clas.dataloaders(df, bs=64)

/anaconda/envs/fastai/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [65]:
def get_doc_term_matrix(train_ds, n_terms):
    """
    inputs: train_ds, n_terms. 
    output: CSR format as scipy.sparse.csr.csr_matrix object. 
    """
    values, column_indices, row_pointer = [], [], []
    row_pointer.append(0)

    for _, (doc, label) in enumerate(train_ds):
        feature_counter = Counter(doc.data.numpy())
        column_indices.extend(feature_counter.keys())
        values.extend(feature_counter.values())
         # Tack on N (number of nonzero elements in the matrix) to the end of the row_pointer array
        row_pointer.append(len(values))

    return scipy.sparse.csr_matrix((values, column_indices, row_pointer),
                                    shape=(len(row_pointer) - 1, n_terms),
                                    dtype=int)

In [66]:
trn_term_doc = get_doc_term_matrix(dls.train_ds, len(dls.vocab.itemgot()[0]))

#### Getting data for spreadsheet: 
To ensure manageable, just get 40 shortest reviews.

In [67]:
inds = np.asarray(trn_term_doc.todense())
inds = np.argpartition(np.count_nonzero(inds, 1), 40, axis=0)[:40]
inds = np.squeeze(np.asarray(inds))


list_text = [dls.train_ds.items.text[i] for i in inds]

# Get counts for all vocab used in our selection of 40 shortest reviews: 
vocab_used = defaultdict(int)

for i in inds:
    for val in dls.train_ds.items.text[i]:
        vocab_used[val] += 1

In [68]:
interesting_inds = [key for key, val in vocab_used.items() if val < 30 and val > 6]
len(interesting_inds)

45

Copy vocab and text of movie reviews directly from here to paste into spreadsheet.

In [70]:
np.array(interesting_inds)

array(['so', 'bad', 'very', 'acting', '\n\n', 'do', 'xxup', "n't", 'one',
       'have', 'seen', 'with', 'are', 'just', 'film', 'but', '!', "'s",
       'story', 'no', 'that', 'as', 'some', 'an', '"', 'great', 'on',
       'if', 'see', 'up', 'will', 'all', 'from', 'me', 'was', 'for',
       'about', 'like', '-', 'not', 'has', 'good', '…', ')', 'out'],
      dtype='<U6')

In [73]:
np.array([" ".join(sentence) for sentence in list_text])[:2]

array(["xxbos xxmaj this movie is so bad , i knew how it ends right after this little girl killed the first person . xxmaj very bad acting very bad plot very bad movie \n\n do yourself a favour and xxup do n't watch it 1 / 10",
       'xxbos xxmaj this is by far one of the worst movies i have ever seen , the poor special effects along with the poor acting are just a few of the things wrong with this film . i am fan of the first two major leagues but this one is lame !'],
      dtype='<U962')

In [74]:
x = trn_term_doc[inds, :]
y = get_y(dls, inds)

In [85]:
stoi(dls.vocab[0], interesting_inds)

array([ 52,  97,  70, 124,  26,  61,   7,  37,  44,  42, 136,  30,  38,
        58,  32,  31,  54,  23,  84,  75,  20,  27,  63,  49,  22, 113,
        35,  59,  88,  71,  96,  46,  51,  90,  25,  28,  56,  50,  24,
        39,  66,  68,  87,  34,  60], dtype=uint8)

In [93]:
from IPython.display import FileLink, FileLinks

path = Path("output")

try: interesting_inds = stoi(dls.vocab[0], interesting_inds)
except Exception: pass

np.savetxt(path/"x_3b.csv", np.asarray(x.todense())[:, interesting_inds].astype(np.uint8),
            delimiter=",", fmt="%i")
FileLink(path/"x_3b.csv")

/home/fastai2/notebooks/fastai_NLP/output/x_3b.csv

In [87]:
np.savetxt(path/"y_3b.csv", y, delimiter=",", fmt="%i")
FileLink(path/"y_3b.csv")

/home/fastai2/notebooks/fastai_NLP/output/y_3b.csv